# Chapter 06 — Autonomous Agents

- Steering behaviors blend desired velocity changes (seek, flee, arrive, wander).
- Boids use three forces: separation, alignment, cohesion; weights tune flock style.
- Limit steering force and speed to keep motion smooth.
- Path following treats the path as a target segment with look-ahead.

Try: Try lowering max_force to see sluggish response or adding noise for wander.

In [ ]:
import randomclass Vector:    def __init__(self, x=0.0, y=0.0):        self.x, self.y = float(x), float(y)    def __add__(self, other):        return Vector(self.x + other.x, self.y + other.y)    def __sub__(self, other):        return Vector(self.x - other.x, self.y - other.y)    def __mul__(self, k):        return Vector(self.x * k, self.y * k)    __rmul__ = __mul__    def mag(self):        return (self.x ** 2 + self.y ** 2) ** 0.5    def normalize(self):        m = self.mag()        if m == 0:            return Vector()        return self * (1.0 / m)    def limit(self, max_mag):        m = self.mag()        if m > max_mag:            return self.normalize() * max_mag        return self    def __repr__(self):        return f"Vector({self.x:.2f}, {self.y:.2f})"class Agent:    def __init__(self, pos):        self.pos = Vector(*pos)        self.vel = Vector(random.uniform(-1, 1), random.uniform(-1, 1)).limit(2)        self.max_speed = 2.5        self.max_force = 0.15    def seek(self, target):        desired = (target - self.pos).normalize() * self.max_speed        steer = (desired - self.vel).limit(self.max_force)        return steer    def apply(self, force):        self.vel = self.vel + force        self.vel = self.vel.limit(self.max_speed)        self.pos = self.pos + self.velagent = Agent((0, 0))target = Vector(10, 0)for i in range(8):    steer = agent.seek(target)    agent.apply(steer)    print(f"step {i}: pos={agent.pos} vel={agent.vel}")